In [1]:
import os
import gc

import pandas as pd
import numpy as np

import torch

from modelscope import snapshot_download
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import SFTTrainer
from peft import get_peft_model, LoraConfig, TaskType

from tqdm import tqdm



2024-03-31 15:15:21,823 - modelscope - INFO - PyTorch version 2.2.1 Found.
2024-03-31 15:15:21,826 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2024-03-31 15:15:21,866 - modelscope - INFO - Loading done! Current index file version is 1.13.1, with md5 b5a2c5fe01f7460b3e700a8ce7e6fc94 and a total number of 972 components indexed
/root/miniconda3/envs/trans/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
gc.collect()
torch.cuda.empty_cache()
torch.cuda.is_available()

True

In [3]:
DATA_DIRECTORY = r'/root/autodl-tmp/AIST4010-Cantonese-Translator-Data/'

def load_cantonese_wiki():
    wiki_lines = []
    def load_cantonese_wiki_file(filename='wiki_00'):
        with open(os.path.join(DATA_DIRECTORY, 'Cantonese-Wiki/text', filename), 'r') as f:
            lines = f.readlines()
            return lines
    for file in os.listdir(os.path.join(DATA_DIRECTORY, 'Cantonese-Wiki/text')):
        curr_lines = load_cantonese_wiki_file(file)
        wiki_lines.extend(curr_lines)
    
    return wiki_lines

def load_openrice_reviews():
    with open(os.path.join(DATA_DIRECTORY, 'openrice/openrice.txt'), 'r') as f:
        lines = f.readlines()
        return lines

yue_wiki_lines = load_cantonese_wiki()
openrice_lines = load_openrice_reviews()

print(len(yue_wiki_lines))
print(len(openrice_lines))

mono_dataset = Dataset.from_dict({
    'text': yue_wiki_lines + openrice_lines
})

print(len(mono_dataset))

#print mean sentence length
sentence_lengths = [len(sentence) for sentence in mono_dataset['text']]
print(np.mean(sentence_lengths))
print(np.sum(sentence_lengths))


1116717
11997
1128714
27.67120457440946
31232876


In [4]:
model_path=r'/root/autodl-tmp/01ai/Yi-6B-Chat'
model_dir = snapshot_download('01ai/Yi-6B-Chat', cache_dir='/root/autodl-tmp', revision='master')

In [5]:
base_tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True, padding_side='right', max_length=512, return_tensors='pt')
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True, padding_side='right', max_length=512, return_tensors='pt')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


# Since transformers 4.35.0, the GPT-Q/AWQ model can be loaded using AutoModelForCausalLM.
base_model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map=device,
    torch_dtype='auto',
)


# # Prompt content: "hi"
# messages = [
#     {"role": "user", "content": "hi"}
# ]


# input_ids = tokenizer.apply_chat_template(conversation=messages, tokenize=True, add_generation_prompt=True, return_tensors='pt')
# output_ids = model.generate(input_ids.to('cuda'))
# response = tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True)

# # Model response: "Hello! How can I assist you today?"
# print(response)

Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.27s/it]


In [6]:
# Prompt content: "hi"
messages = [
    {"role": "user", "content": "hi"}
]


input_ids = tokenizer.apply_chat_template(conversation=messages, tokenize=True, add_generation_prompt=True, return_tensors='pt')
output_ids = base_model.generate(input_ids.to('cuda'))
response = tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True)

# Model response: "Hello! How can I assist you today?"
print(response)

Hello! It looks like you're having a conversation. How can I help you today?


In [7]:
lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules = ["k_proj", "q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)
peft_model = get_peft_model(base_model, 
                            lora_config)

peft_model.print_trainable_parameters()

trainable params: 17,825,792 || all params: 6,078,861,312 || trainable%: 0.293242288071467


In [ ]:
yue_tokenizer = AutoTokenizer.from_pretrained('/root/new_tokenizer', use_fast=True, padding_side='right', max_length=512, return_tensors='pt')

In [ ]:
print(tokenizer("嗌呃畀啲嘢噃")['input_ids'])
print(base_tokenizer("嗌呃畀啲嘢噃")['input_ids'])
print(tokenizer.tokenize("嗌呃畀啲嘢噃"))
print(base_tokenizer.tokenize("嗌呃畀啲嘢噃"))
print(tokenizer("Good morning")['input_ids'])
print(base_tokenizer("Good morning")['input_ids'])

[64617, 534, 450, 436, 536, 454, 433, 534, 454, 483, 534, 457, 467, 534, 458, 436]
[59568, 534, 456, 445, 534, 450, 436, 536, 454, 433, 534, 454, 483, 534, 457, 467, 534, 458, 436]
['▁嗌', '<0xE5>', '<0x91>', '<0x83>', '<0xE7>', '<0x95>', '<0x80>', '<0xE5>', '<0x95>', '<0xB2>', '<0xE5>', '<0x98>', '<0xA2>', '<0xE5>', '<0x99>', '<0x83>']
['▁', '<0xE5>', '<0x97>', '<0x8C>', '<0xE5>', '<0x91>', '<0x83>', '<0xE7>', '<0x95>', '<0x80>', '<0xE5>', '<0x95>', '<0xB2>', '<0xE5>', '<0x98>', '<0xA2>', '<0xE5>', '<0x99>', '<0x83>']
[6076, 4040]
[6076, 4040]


In [ ]:
# bleu = evaluate.load('bleu')

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    print(predictions.shape, labels.shape)
    return {"bleu": bleu(predictions, labels)}

In [ ]:
peft_model.resize_token_embeddings(len(tokenizer))

Embedding(65170, 4096)

In [ ]:
training_args = TrainingArguments(
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=3,
    logging_steps=100,
    output_dir="/root/peft_model"
)

trainer = SFTTrainer(
    peft_model,
    args=training_args,
    train_dataset= abc_train_set,
    eval_dataset= abc_test_set,
    tokenizer=tokenizer,
    formatting_func=formatting_prompts_func,
    compute_metrics=compute_metrics,
)
trainer.train()

Map:   0%|          | 0/13026 [00:00<?, ? examples/s]

Map: 100%|██████████| 1448/1448 [00:00<00:00, 4026.51 examples/s]


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
trainer.model.save_pretrained("/root/peft_model")

/root/miniconda3/envs/trans/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /root/autodl-tmp/01ai/Yi-6B-Chat - will assume that the vocabulary was not modified.
  warnings.warn(


In [ ]:
# #get random data from test dataset
# for i in range(5):
#     example = abc_test_set[i]
#     print(example)
#     text1 = f"""Translate the following words into Cantonese: 
#         {example['en']}
#         """
#     text2 = f"""Translate the following words into English:
#         {example['yue']}
#         """
#     texts = [text1, text2]
#     for text in texts:
#         messages = [
#             {"role": "user", "content": text}
#         ]
#         print(messages)
#         #print model outputs for base_model and peft_model
#         base_input_ids = base_tokenizer.apply_chat_template(conversation=messages, tokenize=True, add_generation_prompt=True, return_tensors='pt')
#         peft_input_ids = tokenizer.apply_chat_template(conversation=messages, tokenize=True, add_generation_prompt=True, return_tensors='pt')
#         print("Base ID:", base_input_ids)
#         print("Base Input:", base_tokenizer.decode(base_input_ids[base_input_ids.shape[1]:], skip_special_tokens=True))
#         print("PEFT ID:", peft_input_ids)
#         print("PEFT Input:", tokenizer.decode(peft_input_ids[peft_input_ids.shape[1]:], skip_special_tokens=True))
#         print(peft_input_ids)
#         base_output_ids = base_model.generate(base_input_ids.to('cuda'), max_new_tokens=100)
#         peft_output_ids = peft_model.generate(peft_input_ids.to('cuda'), max_new_tokens=100)
#         print(base_output_ids.shape, peft_output_ids.shape)
#         print("Base model: ", base_tokenizer.decode(base_output_ids[0][base_input_ids.shape[1]:], skip_special_tokens=True))
#         print("Fine-tuned: ", tokenizer.decode(peft_output_ids[0][peft_input_ids.shape[1]:], skip_special_tokens=True))


In [ ]:
print(pd.DataFrame(trainer.state.log_history))

     loss  grad_norm  learning_rate  epoch  step  train_runtime  \
0  1.4801   0.750000       0.000667   0.03   100            NaN   
1  1.0638   0.558594       0.000333   0.06   200            NaN   
2  1.0345   0.507812       0.000000   0.09   300            NaN   
3     NaN        NaN            NaN   0.09   300        66.8764   

   train_samples_per_second  train_steps_per_second    total_flos  train_loss  
0                       NaN                     NaN           NaN         NaN  
1                       NaN                     NaN           NaN         NaN  
2                       NaN                     NaN           NaN         NaN  
3                    35.887                   4.486  8.621312e+15    1.192813  
